In [1]:
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [ ]:
!ls

drive  sample_data


In [ ]:
# Situacao 1: Ticker esta presente na lista de validos, mas nao tem .csv presente na pasta de dados

# Decisao 1: Verificar necessidade de incluir obrigatoriamente a informacao do exchange

In [29]:
import pandas as pd
import datetime
import os.path
from pathlib import Path

import re # built-in method for splitting strings
import random

In [3]:
!pip install findatapy

     |████████████████████████████████| 1.7MB 6.0MB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
     |████████████████████████████████| 81kB 8.9MB/s 
     |████████████████████████████████| 491kB 46.9MB/s 
     |████████████████████████████████| 81kB 9.4MB/s 
     |████████████████████████████████| 71kB 8.7MB/s 
     |████████████████████████████████| 61kB 7.3MB/s 
     |████████████████████████████████| 204kB 49.5MB/s 
     |████████████████████████████████| 71kB 8.4MB/s 
     |████████████████████████████████| 1.3MB 44.5MB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
     |████████████████████████████████| 51kB 6.2MB/s 
     |████████████████████████████████| 245kB 46.8MB/s 
     |████████████████████████████████| 143kB 40.0MB/s 
     |████████████████████████████████| 296kB 39.1MB/s 
     |████████████████████████████████| 2.6MB 47.3MB/s 
     |████████████████████████████████| 61kB 6.7MB/s 
     |████████████████████████████████| 102kB 10.7MB/s 
  Created w

In [4]:
from findatapy.market import Market, MarketDataRequest, MarketDataGenerator


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [27]:
#foo = pd.read_csv("/content/drive/My Drive/Finance/data/aapl.csv")
PATH = "/content/drive/My Drive/Finance/Data"
YAHOO_PATH = "/content/drive/My Drive/Finance/Data/fromYahooFinance"
YAHOO_STOCKS_PATH = "/content/drive/My Drive/Finance/Data/fromYahooFinance/Stocks"

MONTH_CONVERTER = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 
                   5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 
                   9: 'Sep', 10: 'Oct', 11: 'Nov', 12: 'Dec'}


# Functions and Classes

In [25]:
def saveStocksToCSV(df, name, data_source, exchange=None):
    if data_source == 'yahoo':
        if exchange == None:
            filepath = "{}/{}.csv".format(YAHOO_STOCKS_PATH, name)
            output_dir = Path(YAHOO_STOCKS_PATH)
            output_dir = "{}/no_exchange_information".format(YAHOO_STOCKS_PATH)
        else:
            filepath = "{}/{}/{}.csv".format(YAHOO_STOCKS_PATH, exchange, name)
            output_dir = Path("{}/{}".format(YAHOO_STOCKS_PATH, exchange))
    try:
        output_dir.mkdir(parents=True, exist_ok=True)
        df.to_csv(filepath)
        print("Saved {}.csv successfully".format(name))
    except Exception as e:
        print("Error '{}' occured while saving '{}'".format(e, filepath))


def getStocksHistoricalData(vendor_ticker, data_source, exchange=None):
    # Fetch data
    md_request = MarketDataRequest(
        start_date = 'Jan 1 1950',
        data_source=data_source, 
        tickers=vendor_ticker,
        fields=['open', 'high', 'low', 'close', 'adjclose', 'volume'],
        vendor_tickers=vendor_ticker,
        vendor_fields=['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'])

    df = market.fetch_market(md_request)
    return df


def saveStocksHistoricalData(vendor_ticker, data_source, exchange=None):
   # Fetch data
    md_request = MarketDataRequest(
        start_date = 'Jan 1 1950',
        data_source=data_source, 
        tickers=vendor_ticker,
        fields=['open', 'high', 'low', 'close', 'adjclose', 'volume'],
        vendor_tickers=vendor_ticker,
        vendor_fields=['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'])

    df = market.fetch_market(md_request)
    saveStocksToCSV(df=df, name=vendor_ticker, data_source=data_source, exchange=exchange)


def addToTickerList(ticker, valid, data_source, exchange=None):
    aux = "valid" if valid else "invalid"
    if data_source == 'yahoo':
        stock_list = "yahoo_valid_stocks.csv" if valid else "yahoo_invalid_stocks.csv"
        f_path = "{}/{}".format(YAHOO_VALID_STOCKS_FILE, stock_list)
    
    if os.path.isfile(f_path):
        df = pd.read_csv(f_path, index_col=0)
        if ticker not in df.index.values:
            new_entry = pd.DataFrame([[ticker, exchange]], columns=['ticker', 'exchange']).set_index('ticker')
            df = df.append(new_entry)
            df.to_csv(f_path, mode='w', index=True)
            print("Ticker {} 'added' to {} tickers. (not verified)".format(ticker, aux))
        else:
            print("Ticker {} is already present in {} tickers.".format(ticker, aux))
    else:
        try:
            pd.DataFrame(columns=['ticker', 'exchange']).set_index('ticker').to_csv(f_path, index=True)
            print ("File not exist. New file created.")
            addToTickerList(ticker, valid, data_source, exchange)
        except:
            print ("File not exist. Could not create new file.")


def removeFromTickerList(ticker, valid, data_source, exchange=None):
    aux = "valid" if valid else "invalid"
    if data_source == 'yahoo':
        stock_list = "yahoo_valid_stocks.csv" if valid else "yahoo_invalid_stocks.csv"
        f_path = "{}/{}".format(YAHOO_VALID_STOCKS_FILE, stock_list)
    
    if os.path.isfile(f_path):
        df = pd.read_csv(f_path, index_col=0)
        if ticker in df.index.values:
            #new_entry = pd.DataFrame([[ticker, None]], columns=['ticker', 'exchange']).set_index('ticker')
            #df = df.append(new_entry)
            df.drop(index=ticker, inplace=True)
            df.to_csv(f_path, mode='w', index=True)
            print("Ticker {} 'removed' to {} tickers. (not verified)".format(ticker, aux))
        else:
            print("Ticker {} is not present in {} tickers.".format(ticker, aux))

    else:
        try:
            pd.DataFrame(columns=['ticker', 'exchange']).set_index('ticker').to_csv(f_path, index=True)
            print ("File not exist. New file created.")
            addToTickerList(ticker, valid, data_source, exchange)
        except:
            print ("File not exist. Could not create new file.")

      
def verifyTickerValidity(tickerList, data_source, exchange=None):
  """
  Request historical data for a list of vendor_tickers. If successful, add
  the ticker to valid list or invalid list otherwise.
  """
  for vendor_ticker in tickerList:
      df = getStocksHistoricalData(vendor_ticker, data_source, exchange)
      if df.empty:
          addToTickerList(ticker=vendor_ticker, valid=False, data_source=data_source, exchange=exchange)
      else:
          addToTickerList(ticker=vendor_ticker, valid=True, data_source=data_source, exchange=exchange)
          saveStocksToCSV(df=df, name=vendor_ticker, data_source=data_source, exchange=exchange)


def updateStockData(vendor_ticker):
    # Get date of the most recent data entry from csv file
    df = pd.read_csv(PATH + "/{}.csv".format(vendor_ticker), index_col=0)
    print("Data mais recente: {}".format(df.index[-1]))
    last_date = df.index[-1]
    split_date = re.split('-', last_date)
    year, month, day = int(split_date[0]), int(split_date[1]), int(split_date[2])
    start_date = "{} {} {}".format(MONTH_CONVERTER[month], day, year)

    # Fetch data
    md_request = MarketDataRequest(
        start_date=start_date,
        data_source='yahoo', 
        tickers=vendor_ticker,
        fields=['open', 'high', 'low', 'close', 'adjclose', 'volume'],
        vendor_tickers=vendor_ticker,
        vendor_fields=['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'])

    new_df = market.fetch_market(md_request)

    # "Append" the new data in the csv file
    df = pd.concat([df, new_df], axis=0)
    df.drop_duplicates(inplace=True)
    return df

In [30]:
verifyTickerValidity(vendor_ticker_list, 'yahoo', exchange)

2020-11-13 01:16:06,984 - findatapy.market.datavendorweb - INFO - Request Yahoo data
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AIB.IR: No data found for this date range, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- AIB.IR: No data found for this date range, symbol may be delisted
2020-11-13 01:16:07,179 - findatapy.market.datavendorweb - INFO - Completed request from Yahoo.
2020-11-13 01:16:07,186 - findatapy.market.ioengine - WARNING - Couldn't push MarketDataRequest_477__abstract_curve_key-None__category-None__category_key-backtest_default-cat_yahoo_daily_NYC__cut-NYC__data_source-yahoo__environment-backtest__expiry_date-NaT__fields-open_high_low_close_adjclose_volume__finish_date-2020-11-13 00:23:31_207106__freq-daily__freq_mult-1__gran_freq-None__push_to_cache-True__resample-None__resample_how-last__start_date-1950-01-01 00:00:00__tickers-AIB_IR__trade_side-trade__v

In [ ]:
#verifyYahooTickerValidity(vendor_ticker_list, 'ISE')

# Fake "main"

In [23]:
market = Market(market_data_generator=MarketDataGenerator())


In [ ]:


#stockDict = [{'ticker': 'Apple', 'vendor_ticker': 'aapl'}]
#ticker = 'Lojas Americanas S.A.' #'Apple'
vendor_ticker = 'aapl'
#ticker = br_stuff['Name'].values.tolist()[:100] #'Apple'
#vendor_ticker = br_stuff['Ticker'].values.tolist()[:100] #'aapl'

md_request = MarketDataRequest(
    start_date='Jun 1 2020 01:33',
    finish_date = 'Nov 2 2020',
    data_source='yahoo',
    #category = "fundamental",
    tickers=vendor_ticker,
    fields=['open', 'high', 'low', 'close', 'adjclose', 'volume'],
    vendor_tickers=vendor_ticker,
    vendor_fields=['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume'])

df = market.fetch_market(md_request)
#df = pd.read_csv(PATH + "/{}.csv".format(vendor_ticker), index_col=0)
#saveStocksHistoricalData(vendor_ticker, exchange="SAO")

#df_updated = updateStockData(vendor_ticker)
#saveStocksToCSV(df_updated, vendor_ticker)

2020-11-11 17:58:41,340 - findatapy.market.datavendorweb - INFO - Request Yahoo data
2020-11-11 17:58:41,585 - numexpr.utils - INFO - NumExpr defaulting to 2 threads.
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
2020-11-11 17:58:41,776 - findatapy.market.datavendorweb - INFO - Completed request from Yahoo.
2020-11-11 17:58:41,811 - findatapy.market.ioengine - WARNING - Couldn't push MarketDataRequest_466__abstract_curve_key-None__category-None__category_key-backtest_default-cat_yahoo_daily_NYC__cut-NYC__data_source-yahoo__environment-backtest__expiry_date-NaT__fields-open_high_low_close_adjclose_volume__finish_date-2020-11-02 00:00:00__freq-daily__freq_mult-1__gran_freq-None__push_to_cache-True__resample-None__resample_how-last__start_date-2020-06-01 01:33:00__tickers-aapl__trade_side-trade__vendor_fields-Open_High_Low_Close_Adj Close_Volume__vendor_tickers-aapl to Redis: Error 111 connecting t

# Experiment 1

In [8]:
#datetime.datetime.now().year
#random.randint(50, 150)

In [9]:
stuff_path = "/content/drive/My Drive/Finance/Yahoo Ticker Symbols - September 2017.xlsx"
stuff = pd.read_excel(stuff_path, sheet_name="Stock", header=3, usecols="A:E")
stuff

,Ticker,Name,Exchange,Category Name,Country
0,OEDV,"Osage Exploration and Development, Inc.",PNK,NaN,USA
1,AAPL,Apple Inc.,NMS,Electronic Equipment,USA
2,BAC,Bank of America Corporation,NYQ,Money Center Banks,USA
3,AMZN,"Amazon.com, Inc.",NMS,Catalog & Mail Order Houses,USA
4,T,AT&T Inc.,NYQ,Telecom Services - Domestic,USA
...,...,...,...,...,...
106323,G5N.SG,"Groupon Inc. Reg. Shares DL-,00",STU,NaN,Germany
106324,N1H.AX,N1 Holdings Limited,ASX,Mortgage Investment,Australia
106325,1K2.BE,"K2M GROUP HLDGS DL-,001",BER,NaN,Germany
106326,2RM.BE,"RE/MAX HLDGS A DL -,0001",BER,NaN,Germany


In [15]:
#stuff['Exchange'].value_counts().tail(10)
exchange = 'ISE'
print(stuff[stuff['Exchange'] == exchange].shape)
stuff[stuff['Exchange'] == exchange].head()

(55, 5)


,Ticker,Name,Exchange,Category Name,Country
1581,AIB.IR,AIB.IR,ISE,NaN,Ireland
6056,DQ5.IR,CPL Resources plc,ISE,NaN,Ireland
13000,JEV.IR,JEV.IR,ISE,NaN,Ireland
13288,IRES.IR,Irish Residential Properties REIT plc,ISE,NaN,Ireland
13557,MSTY.IR,Mainstay Medical International plc,ISE,NaN,Ireland


In [16]:
exchange_df = stuff[stuff['Exchange'] == exchange] #SAO
#ticker = br_stuff['Name'].values.tolist() #'Apple'
vendor_ticker_list = exchange_df['Ticker'].values.tolist()[:100] #'aapl'
vendor_ticker_list

['AIB.IR',
 'DQ5.IR',
 'JEV.IR',
 'IRES.IR',
 'MSTY.IR',
 'SK3.IR',
 'KRZ.IR',
 'FAC.IR',
 'BIR.IR',
 'ORQ.IR',
 'T7O.IR',
 'MLC.IR',
 'APGN.IR',
 'HSW.IR',
 'YZA.IR',
 'AIB1.IR',
 'RY4C.IR',
 'ZAZ.IR',
 'FKV1.IR',
 'DOY.IR',
 'DOP.IR',
 'PPB.IR',
 'DD7E.IR',
 'DD7D.IR',
 'GN1.IR',
 'CRG.IR',
 'CRO.IR',
 'DD8B.IR',
 'DD8A.IR',
 'DLE.IR',
 'DHG.IR',
 'WLG.IR',
 'KJY.IR',
 'DQ7.IR',
 'EG7.IR',
 'EG5A.IR',
 'FQ3.IR',
 'GAME.IR',
 'GCC.IR',
 'HBRN.IR',
 'GL9.IR',
 'OVXA.IR',
 'GYQ.IR',
 'P8ET.IR',
 'IPDC.IR',
 '3MP.IR',
 'KRX.IR',
 '8GW.IR',
 'OIZ.IR',
 'PZQA.IR',
 'IL0A.IR',
 'IJG.IR',
 'IR5B.IR',
 'MIO.IR',
 'TQW.IR']